# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [129]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [130]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,50.94,64,100,6.06,US,1668999658
1,1,bambous virieux,-20.3428,57.7575,76.21,71,19,8.79,MU,1668999659
2,2,scituate,42.1959,-70.7259,30.49,43,39,8.01,US,1668999659
3,3,atuona,-9.8000,-139.0333,77.49,72,32,9.64,PF,1668999660
4,4,tasiilaq,65.6145,-37.6368,37.56,81,100,14.90,GL,1668999661


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [131]:
#
cities = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Configure the map plot
#fig = gmaps.figure(center=(46.0, -5.0), zoom_level=3)
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})

# Display the map
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='1200px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [132]:
# Narrow down cities that fit criteria and drop any results with null values
dream_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70)
                                    & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)
                                    ]
dream_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,santa maria,-29.6842,-53.8069,72.70,68,0,4.61,BR,1668999272
36,36,busselton,-33.6500,115.3333,71.13,39,0,5.44,AU,1668999687
251,251,umm lajj,25.0213,37.2685,79.57,45,0,9.35,SA,1668999841
381,381,saint-pierre,-21.3393,55.4781,73.08,56,0,6.91,RE,1668999936
496,496,cap malheureux,-19.9842,57.6142,76.46,69,0,5.99,MU,1669000020


### Step 3: Create a new DataFrame called `hotel_df`.

In [133]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = dream_cities_df.copy(deep=True)
#hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
#hotel_df["Hotel Lat"] = ""
#hotel_df["Hotel Lng"] = ""
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,24,santa maria,-29.6842,-53.8069,72.70,68,0,4.61,BR,1668999272,
36,36,busselton,-33.6500,115.3333,71.13,39,0,5.44,AU,1668999687,
251,251,umm lajj,25.0213,37.2685,79.57,45,0,9.35,SA,1668999841,
381,381,saint-pierre,-21.3393,55.4781,73.08,56,0,6.91,RE,1668999936,
496,496,cap malheureux,-19.9842,57.6142,76.46,69,0,5.99,MU,1669000020,


### Step 4: For each city, use the Google Places API to find the first hotel located within 5,000 metres of your coordinates.

In [134]:
# Set parameters to search for a hotel
params = {
    "keyword": "hotel",
    "radius": 5000,
    "key": gkey
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santa maria - nearest hotel: P1 Hotel
busselton - nearest hotel: Esplanade Hotel
umm lajj - nearest hotel: HP Red Sea Hotel
saint-pierre - nearest hotel: Villa Delisle
cap malheureux - nearest hotel: Paradise Cove Boutique Hotel


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,24,santa maria,-29.6842,-53.8069,72.70,68,0,4.61,BR,1668999272,P1 Hotel
36,36,busselton,-33.6500,115.3333,71.13,39,0,5.44,AU,1668999687,Esplanade Hotel
251,251,umm lajj,25.0213,37.2685,79.57,45,0,9.35,SA,1668999841,HP Red Sea Hotel
381,381,saint-pierre,-21.3393,55.4781,73.08,56,0,6.91,RE,1668999936,Villa Delisle
496,496,cap malheureux,-19.9842,57.6142,76.46,69,0,5.99,MU,1669000020,Paradise Cove Boutique Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [136]:
hotel_locations = name_address["results"][0]["location"]
hotel_ratings = name_address["results"][0]["rating"]

# Configure the map plot
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})

# Display the map
heat_layer = gmaps.heatmap_layer(hotel_locations, weights=hotel_ratings, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

KeyError: 'location'